# Aviation Accidents Analysis

You are part of a consulting firm that is tasked to do an analysis of commercial and passenger jet airline safety. The client (an airline/airplane insurer) is interested in knowing what types of aircraft (makes/models) exhibit low rates of total destruction and low likelihood of fatal or serious passenger injuries in the event of an accident. They are also interested in any general variables/conditions that might be at play. Your analysis will be based off of aviation accident data accumulated from the years 1948-2023. 

Our client is only interested in airplane makes/models that are professional builds and could potentially still be active. Assume a max lifetime of 40 years for a make/model retirement and make sure to filter your data accordingly (i.e. from 1983 onwards). They would also like separate recommendations for small aircraft vs. larger passenger models. **In addition, make sure that claims that you make are statistically robust and that you have enough samples when making comparisons between groups.**


In this summative assessment you will demonstrate your ability to:
- **Use Pandas to load, inspect, and clean the dataset appropriately.**
- **Transform relevant columns to create measures that address the problem at hand.**
- conduct EDA: visualization and statistical measures to systematically understand the structure of the data
- recommend a set of airplanes and makes conforming to the client's request and identify at least *two* factors contributing to airplane safety. You must provide supporting evidence (visuals, summary statistics, tables) for each claim you make.

### Make relevant library imports

In [383]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

## Data Loading and Inspection

### Load in data from the relevant directory and inspect the dataframe.
- inspect NaNs, datatypes, and summary statistics

In [384]:
df = pd.read_csv('/Users/mikeb/Documents/Flatiron/AD02/lab/dsc-course0-m8-lab/data/AviationData.csv', 
                 encoding='latin-1',
                 low_memory=False)

In [385]:
print(df.head(100))

          Event.Id Investigation.Type Accident.Number  Event.Date  \
0   20001218X45444           Accident      SEA87LA080  1948-10-24   
1   20001218X45447           Accident      LAX94LA336  1962-07-19   
2   20061025X01555           Accident      NYC07LA005  1974-08-30   
3   20001218X45448           Accident      LAX96LA321  1977-06-19   
4   20041105X01764           Accident      CHI79FA064  1979-08-02   
..             ...                ...             ...         ...   
95  20020917X01625           Accident      ANC82DA015  1982-01-15   
96  20020917X02096           Accident      FTW82FA041  1982-01-16   
97  20020917X02487           Accident      NYC82DA023  1982-01-16   
98  20020917X02321           Accident      LAX82FVG14  1982-01-17   
99  20020917X02306           Accident      LAX82FUQ08  1982-01-17   

            Location        Country   Latitude   Longitude Airport.Code  \
0    MOOSE CREEK, ID  United States        NaN         NaN          NaN   
1     BRIDGEPORT, CA 

In [386]:
headers = df.columns.tolist()

for value in headers:
    print(value)

Event.Id
Investigation.Type
Accident.Number
Event.Date
Location
Country
Latitude
Longitude
Airport.Code
Airport.Name
Injury.Severity
Aircraft.damage
Aircraft.Category
Registration.Number
Make
Model
Amateur.Built
Number.of.Engines
Engine.Type
FAR.Description
Schedule
Purpose.of.flight
Air.carrier
Total.Fatal.Injuries
Total.Serious.Injuries
Total.Minor.Injuries
Total.Uninjured
Weather.Condition
Broad.phase.of.flight
Report.Status
Publication.Date


In [387]:
unique_values = sorted(df['Make'].astype(str).unique())

# Print each value on a new line (column-style)
for value in unique_values:
    print(value)

107.5 Flying Corporation
1200
177MF LLC
1977 Colfer-chan
1st Ftr Gp
2000 Mccoy
2001 Mcgirl
2003 Nash
2007 Savage Air LLC
2021FX3 LLC
3XTRIM
5 RIVERS LLC
67 Flying Dutchman
737
777
781569 INC
85 Manista
A Pair Of Jacks
A. H. Gettings
A. Le Francois
A. Schleicher GMBH & Co.
AAA AIRCRAFT LLC
AARDEMA ROBERT JOHN
AB SPORTINE AVIACIJA
AB Sportine Aviacija
ABBETT GERRY
ABBEY VICTOR
ACKLAND JEFFREY DEAN
ACRO
ADAMS
ADAMS BALLOONS LLC
ADAMS DENNIS ALLEN
ADAMS DONALD L
ADAMS JOHN R JR
ADRIAN GEORGE W
AERMACCHI
AERO ADVENTURE
AERO AT SP ZOO
AERO COMMANDER
AERO SMART SOLUTIONS INC
AERO SP Z O O
AERO TEK INC.
AERO VODOCHODY
AERO-ACE
AEROFAB INC
AEROFAB INC.
AEROLITE
AEROMOT
AERONCA
AEROPRAKT
AEROPRAKT MANUFACTURING SP ZOO
AEROPRO CZ
AEROPRO CZ S R O
AEROS
AEROS LTD
AEROS LTD/SKYRANGER AIRCRAFT
AEROSPATIALE
AEROSPATIALE ALENIA
AEROSPOOL
AEROSPORT LTD
AEROSTAR
AEROSTAR ACFT CORP OF TEXAS
AEROSTAR AIRCRAFT CORPORATION
AEROSTAR INTERNATIONAL
AEROSTAR INTERNATIONAL INC
AEROSTAR S A
AEROTECHNIK
AEROTEK
AE

## Data Cleaning

### Filtering aircrafts and events

We want to filter the dataset to include aircraft that the client is interested in an analysis of:
- inspect relevant columns
- figure out any reasonable imputations
- filter the dataset

In [388]:
# Aircraft.Category:  Client is only interested in airplanes

# Look at all of the possible values

# Column to analyze
col_name = 'Aircraft.Category'

# Count occurrences of each unique value
value_counts = df[col_name].value_counts().sort_index()


# Print each value and its count
for value, count in value_counts.items():
    print(f"{value}: {count}")

Airplane: 27617
Balloon: 231
Blimp: 4
Glider: 508
Gyrocraft: 173
Helicopter: 3440
Powered Parachute: 91
Powered-Lift: 5
Rocket: 1
ULTR: 1
UNK: 2
Ultralight: 30
Unknown: 14
WSFT: 9
Weight-Shift: 161


In [389]:
# Keep only Airplane records
df = df[df['Aircraft.Category'] == 'Airplane']

# Verify
print(df['Aircraft.Category'].value_counts())

Aircraft.Category
Airplane    27617
Name: count, dtype: int64


In [390]:
# Amateur.Built:  Client is only interested in professional builds

# Column to analyze
col_name = 'Amateur.Built'

# Count occurrences of each unique value
value_counts = df[col_name].value_counts().sort_index()


# Print each value and its count
for value, count in value_counts.items():
    print(f"{value}: {count}")

No: 24417
Yes: 3183


In [391]:
# Keep only professional build records
df = df[df['Amateur.Built'] == 'No']

# Verify
print(df['Amateur.Built'].value_counts())

Amateur.Built
No    24417
Name: count, dtype: int64


In [392]:
# Event.Date:  Removing all events older than 40 years ago

# Column to analyze
col_name = 'Event.Date'

# Count occurrences of each unique value
value_counts = df[col_name].value_counts().sort_index()


# Print each value and its count
for value, count in value_counts.items():
    print(f"{value}: {count}")

1979-09-17: 1
1982-01-01: 2
1982-01-02: 8
1982-01-03: 9
1982-01-04: 5
1982-01-05: 8
1982-01-06: 4
1982-01-07: 3
1982-01-08: 8
1982-01-09: 5
1982-01-10: 6
1982-01-11: 4
1982-01-12: 4
1982-01-13: 4
1982-01-15: 6
1982-01-16: 1
1982-01-17: 9
1982-01-18: 5
1982-01-19: 16
1982-01-20: 6
1982-01-21: 8
1982-01-22: 2
1982-01-23: 11
1982-01-24: 5
1982-01-25: 2
1982-01-26: 5
1982-01-27: 5
1982-01-28: 5
1982-01-29: 4
1982-01-30: 11
1982-01-31: 9
1982-02-01: 8
1982-02-02: 4
1982-02-03: 6
1982-02-04: 1
1982-02-05: 3
1982-02-06: 11
1982-02-07: 13
1982-02-08: 9
1982-02-09: 7
1982-02-10: 5
1982-02-11: 3
1982-02-12: 4
1982-02-13: 10
1982-02-14: 11
1982-02-15: 14
1982-02-16: 3
1982-02-17: 8
1982-02-18: 5
1982-02-19: 8
1982-02-20: 10
1982-02-21: 14
1982-02-22: 8
1982-02-23: 5
1982-02-24: 9
1982-02-25: 6
1982-02-26: 4
1982-02-27: 9
1982-02-28: 7
1982-03-01: 5
1982-03-02: 8
1982-03-03: 4
1982-03-04: 8
1982-03-05: 6
1982-03-06: 10
1982-03-07: 7
1982-03-08: 9
1982-03-09: 6
1982-03-10: 9
1982-03-11: 7
1982-03-1

In [393]:
# Convert to datetime 
# df['Event.Date'] = pd.to_datetime(df['Event.Date'], errors='coerce')
df['Event.Date'] = pd.to_datetime(df['Event.Date'], errors='coerce')

# Keep only records after from 1983 and later
df = df[df['Event.Date'].dt.year >= 1983]

print(df.head())

            Event.Id Investigation.Type Accident.Number Event.Date  \
4149  20001214X42478           Incident     LAX83IA149B 1983-03-18   
4150  20001214X42478           Incident     LAX83IA149A 1983-03-18   
4171  20001214X42331           Accident      ATL83FA140 1983-03-20   
4285  20001214X42672           Accident      FTW83LA177 1983-04-02   
5957  20001214X44248           Incident      MIA83IA210 1983-08-21   

             Location        Country Latitude Longitude Airport.Code  \
4149  LOS ANGELES, CA  United States      NaN       NaN          LAX   
4150  LOS ANGELES, CA  United States      NaN       NaN          LAX   
4171   CROSSVILLE, TN  United States      NaN       NaN          NaN   
4285     MCKINNEY, TX  United States      NaN       NaN         TX05   
5957      NORFOLK, VA  United States      NaN       NaN          NaN   

          Airport.Name  ... Purpose.of.flight               Air.carrier  \
4149  LOS ANGELES INTL  ...           Unknown                       NaN

### Cleaning and constructing Key Measurables

Injuries and robustness to destruction are a key interest point for the client. Clean and impute relevant columns and then create derived fields that best quantifies what the client wishes to track. **Use commenting or markdown to explain any cleaning assumptions as well as any derived columns you create.**

**Construct metric for fatal/serious injuries**

*Hint:* Estimate the total number of passengers on each flight. The likelihood of serious / fatal injury can be estimated as a fraction from this.

In [394]:
# Add Total.Fatal.Injuries, Total.Serious.Injuries, Total.Minor.Injuries, and Total.Uninjured fields as a representation of the total number of passengers
# Create a new "Total.Passengers" column

# Columns to sum for total passengers
injury_cols = [
    'Total.Fatal.Injuries',
    'Total.Serious.Injuries',
    'Total.Minor.Injuries',
    'Total.Uninjured'
]

# Replace null Nan values with 0 in all columns
for col in injury_cols:
    df[col] = (df[col]).fillna(0)

# Create a new 'Total.Passengers' colmnn by summing all of the columns
df['Total.Passengers'] = df[injury_cols].sum(axis=1)

# Sort the DataFrame by 'Total.Passengers' descending
df_sorted = df.sort_values(by='Total.Passengers', ascending=True).reset_index(drop=True)

# Check results
# Sort the DataFrame by 'Total.Passengers' descending
# print(df_sorted[injury_cols + ['Total.Passengers']].head(10))
print(df.head(10))

            Event.Id Investigation.Type Accident.Number Event.Date  \
4149  20001214X42478           Incident     LAX83IA149B 1983-03-18   
4150  20001214X42478           Incident     LAX83IA149A 1983-03-18   
4171  20001214X42331           Accident      ATL83FA140 1983-03-20   
4285  20001214X42672           Accident      FTW83LA177 1983-04-02   
5957  20001214X44248           Incident      MIA83IA210 1983-08-21   
5960  20001214X44100           Accident      DCA83AA036 1983-08-21   
6669  20001214X44944           Accident      NYC84LA015 1983-10-28   
6760  20001214X45013           Incident      CHI84IA041 1983-11-08   
6806  20001214X45188           Accident      NYC84LA028 1983-11-13   
7084  20001214X45339           Accident      LAX84LA110 1983-12-22   

                   Location        Country Latitude Longitude Airport.Code  \
4149        LOS ANGELES, CA  United States      NaN       NaN          LAX   
4150        LOS ANGELES, CA  United States      NaN       NaN          LA

**Aircraft.Damage**
- identify and execute any cleaning tasks
- construct a derived column tracking whether an aircraft was destroyed or not.

In [395]:
# Column to analyze
col_name = 'Aircraft.damage'

# Count occurrences of each unique value
value_counts = df[col_name].value_counts().sort_index()


# Print each value and its count
for value, count in value_counts.items():
    print(f"{value}: {count}")

Destroyed: 2316
Minor: 817
Substantial: 16990
Unknown: 97


In [396]:
# Replace Unknown as NaN
df['Aircraft.damage'] = df['Aircraft.damage'].replace(['Unknown'], np.nan)

# Create new boolean column to indicatre if the plane was destroyed
df['Aircraft.destroyed'] = df['Aircraft.damage'] == 'Destroyed'

# Check results
print(df[['Aircraft.damage', 'Aircraft.destroyed']].head(10))

     Aircraft.damage  Aircraft.destroyed
4149           Minor               False
4150           Minor               False
4171       Destroyed                True
4285             NaN               False
5957           Minor               False
5960       Destroyed                True
6669             NaN               False
6760           Minor               False
6806     Substantial               False
7084     Substantial               False


### Investigate the *Make* column
- Identify cleaning tasks here
- List cleaning tasks clearly in markdown
- Execute the cleaning tasks
- For your analysis, keep Makes with a reasonable number (you can put the threshold at 50 though lower could work as well)

In [397]:
# Column to analyze
col_name = 'Make'

# Count occurrences of each unique value sorted by the value
#value_counts = df[col_name].value_counts().sort_index()

# sort by counts in descending order
value_counts = df[col_name].value_counts().sort_values(ascending=False)

# Print each value and its count
for value, count in value_counts.items():
    print(f"{value}: {count}")

CESSNA: 4867
PIPER: 2803
Cessna: 2279
Piper: 1186
BOEING: 1037
BEECH: 1018
Beech: 413
MOONEY: 238
Boeing: 227
CIRRUS DESIGN CORP: 218
AIR TRACTOR INC: 217
AIRBUS: 215
BELLANCA: 158
AERONCA: 149
MAULE: 144
Mooney: 125
EMBRAER: 123
Air Tractor: 117
LUSCOMBE: 95
DEHAVILLAND: 91
CHAMPION: 91
STINSON: 91
AIR TRACTOR: 89
CIRRUS: 80
NORTH AMERICAN: 79
GRUMMAN: 77
DIAMOND AIRCRAFT IND INC: 74
AVIAT AIRCRAFT INC: 72
Maule: 71
Grumman: 70
AERO COMMANDER: 68
Champion: 67
BOMBARDIER INC: 65
MCDONNELL DOUGLAS: 64
SOCATA: 63
TAYLORCRAFT: 61
Bellanca: 61
GRUMMAN ACFT ENG COR-SCHWEIZER: 58
Cirrus: 57
RAYTHEON AIRCRAFT COMPANY: 53
ROCKWELL INTERNATIONAL: 52
Aeronca: 51
GRUMMAN AMERICAN AVN. CORP.: 48
Luscombe: 46
AMERICAN CHAMPION AIRCRAFT: 46
BOMBARDIER: 42
Aviat: 42
FLIGHT DESIGN GMBH: 42
Mcdonnell Douglas: 42
De Havilland: 39
Stinson: 38
AYRES CORPORATION: 38
ERCOUPE: 34
MOONEY AIRCRAFT CORP.: 33
Ayres: 33
Taylorcraft: 32
COSTRUZIONI AERONAUTICHE TECNA: 30
Embraer: 30
DIAMOND: 30
AVIAT: 28
Airbus: 2

In [398]:
# Clean the Make column by trimming whitespace and lowercase noormalzing
# Remove commas as they are making one likely Make value appear as mulitple values
df['Make'] = df['Make'].astype(str).str.strip().str.lower().str.replace(',', '')

# make a list of suffixes identified as also making what are likely the same MAke value appear as multiple values
remove_suffixes = ['inc', 'inc.', 'ltd', 'ltd.', 'llc', 'llc.', 'co', 'co.', 'corp', 'corp.', 'corporation']

# strip the suffixes from the end of the Make column using regex on the value
for suffix in remove_suffixes:
    df['Make'] = df['Make'].str.replace(rf'{suffix}$', '', regex=True).str.strip()

# Check results
print(len(df))
print(df.head())

21447
            Event.Id Investigation.Type Accident.Number Event.Date  \
4149  20001214X42478           Incident     LAX83IA149B 1983-03-18   
4150  20001214X42478           Incident     LAX83IA149A 1983-03-18   
4171  20001214X42331           Accident      ATL83FA140 1983-03-20   
4285  20001214X42672           Accident      FTW83LA177 1983-04-02   
5957  20001214X44248           Incident      MIA83IA210 1983-08-21   

             Location        Country Latitude Longitude Airport.Code  \
4149  LOS ANGELES, CA  United States      NaN       NaN          LAX   
4150  LOS ANGELES, CA  United States      NaN       NaN          LAX   
4171   CROSSVILLE, TN  United States      NaN       NaN          NaN   
4285     MCKINNEY, TX  United States      NaN       NaN         TX05   
5957      NORFOLK, VA  United States      NaN       NaN          NaN   

          Airport.Name  ... Total.Fatal.Injuries Total.Serious.Injuries  \
4149  LOS ANGELES INTL  ...                  0.0                 

In [399]:
# Only keep the records where the Make has at least 50 records

# count occurrences per cleaned Make
make_counts = df['Make'].value_counts()               

# keep Makes with 50+ records
makes_to_keep = make_counts[make_counts >= 50].index  

# Create a new df with the filtered Makes
df_filtered = df[df['Make'].isin(makes_to_keep)].reset_index(drop=True)

# Check results
print(len(df_filtered))
print(df_filtered['Make'].value_counts().head(10))  # top 10 Makes


# we went from 88,889 MAkes in the df to 74,975 in the cleaned df

18118
Make
cessna           7146
piper            3989
beech            1431
boeing           1264
air tractor       432
mooney            363
cirrus design     264
airbus            243
bellanca          219
maule             215
Name: count, dtype: int64


### Inspect Model column
- Get rid of any NaNs.
- Inspect the column and counts for each model/make. Are model labels unique to each make?
- If not, create a derived column that is a unique identifier for a given plane type.

In [400]:
# Column to analyze
col_name = 'Model'

# Count occurrences of each unique value sorted by the value
value_counts = df[col_name].value_counts().sort_index()

# sort by counts in descending order
#value_counts = df[col_name].value_counts().sort_values(ascending=False)

# Print each value and its count
for value, count in value_counts.items():
    print(f"{value}: {count}")

0-1A: 1
0-58A: 1
0-58B: 2
01: 1
1: 1
1.131E: 1
10: 1
100: 17
100 180: 1
100-180: 3
1000: 1
108: 56
108 - 3: 1
108 1: 3
108 3: 2
108-1: 19
108-2: 12
108-3: 18
10A: 3
110: 2
110 Special: 1
110P1: 2
110SP: 1
112: 10
1124: 4
1124A: 2
1125: 2
1125 WESTWIND ASTRA: 1
112A: 7
112TC: 5
112TCA: 2
114: 16
114-B: 2
114A: 1
114B: 1
114TC: 1
11A: 1
11AC: 28
11BC: 1
11CC: 3
11E: 1
12-m: 1
120: 60
120A-1: 1
125-700: 1
12Q: 1
12S: 1
14 19: 1
14-13: 1
14-13-2: 1
14-13-3A: 1
14-19: 2
14-19-2: 3
14-19-3: 2
14-19-3A: 1
140: 117
140A: 10
1413: 1
1419: 1
145: 2
146: 3
150: 179
150 - F: 1
150 - G: 1
1500R: 1
150A: 3
150B: 2
150C: 6
150D: 10
150E: 11
150F: 49
150G: 40
150H: 34
150J: 32
150K: 8
150L: 73
150M: 68
152: 317
152G: 1
152K: 1
15AC: 11
162: 23
162 LSA: 1
164-B: 1
164A: 1
164B: 1
16B: 1
17 30: 7
17 30A: 4
17 31: 1
17 31A: 1
17 31ATC: 1
17-30: 4
17-30A: 29
17-31A: 1
17-31ATC: 3
17-A30: 1
170: 74
170 - B: 1
170 B: 1
170-B: 1
170A: 36
170B: 107
172: 772
172 - F: 1
172 - H: 1
172 - M: 2
172 - N: 2
172 - P:

In [401]:
# Drop rows where 'Model' is NaN or empty

# clean the field first by remove leading and trailing whitespace, lowercase normalize
df['Model'] = df['Model'].astype(str).str.strip().str.lower()  

# drop rows where Model field is NaN
df = df.dropna(subset=['Model'])

# Anaylsis of the Model field identified other non-NaN values that are likely not valid Models, drop those rows as well
unwanted_values = ['unknown', 'unk', 'none', 'unkown'] 
df = df[~df['Model'].isin(unwanted_values)].reset_index(drop=True)



# Check the result
print(df[['Make', 'Model']].head(10))
print("Number of records after dropping NaNs and invalid values from Model:", len(df))

             Make              Model
0        lockheed             l-1011
1          boeing                747
2           piper          pa-28-140
3    de havilland              dhc-6
4         douglas           dc-10-10
5        lockheed  learstar, l-18-56
6  short brothers             sd3-30
7          boeing            727-200
8           beech                c35
9          cessna               180k
Number of records after dropping NaNs and invalid values from Model: 21447


In [402]:
# Inspect Make - Model field to see if there Model values are unique to each Make

# Show the Make and Model fields sorted by Model
unique_pairs = df[['Model', 'Make']].drop_duplicates().sort_values(by=['Model', 'Make']).reset_index(drop=True)

# Print each unique pair on its own line
for make, model in zip(unique_pairs['Model'], unique_pairs['Make']):
    print(f"{make} | {model}")

0-1a | cessna
0-58a | aeronca
0-58b | aeronca
01 | ang
1 | glasair
1.131e | casa
10 | stinson
100 | aero commander
100 | beech
100 | fokker
100 | hawker-beechcraft
100 | omf
100 180 | aero commander
100-180 | aero commander
1000 | clark
108 | stinson
108 | universal stinson
108 - 3 | stinson
108 1 | stinson
108 1 | universal stinson
108 3 | stinson
108-1 | stinson
108-1 | univair aircraft
108-1 | universal stinson
108-2 | stinson
108-2 | universal stinson
108-3 | stinson
10a | stinson
10a | volaircraft
110 | monocoupe
110 special | monocoupe
110p1 | embraer
110sp | monocoupe
112 | aero commander
112 | rockwell
112 | rockwell international
1124 | iai
1124 | israel aircraft industries
1124a | israel aircraft industries
1125 | iai
1125 westwind astra | israel aircraft industries
112a | aero commander
112a | rockwell
112a | rockwell international
112tc | rockwell
112tc | rockwell international
112tca | rockwell
112tca | rockwell international
114 | commander
114 | commander aircraft
114 | 

In [403]:
# Since the Models are not always distinct to a Make, need to add new "Make.model" field that combines Make and Model into one field
df['Make.Model'] = df['Make'] + ' -- ' + df['Model']

# Check results
print(df[['Make', 'Model', 'Make.Model']].head(10))

             Make              Model                     Make.Model
0        lockheed             l-1011             lockheed -- l-1011
1          boeing                747                  boeing -- 747
2           piper          pa-28-140             piper -- pa-28-140
3    de havilland              dhc-6          de havilland -- dhc-6
4         douglas           dc-10-10            douglas -- dc-10-10
5        lockheed  learstar, l-18-56  lockheed -- learstar, l-18-56
6  short brothers             sd3-30       short brothers -- sd3-30
7          boeing            727-200              boeing -- 727-200
8           beech                c35                   beech -- c35
9          cessna               180k                 cessna -- 180k


### Cleaning other columns
- there are other columns containing data that might be related to the outcome of an accident. We list a few here:
- Engine.Type
- Weather.Condition
- Number.of.Engines
- Purpose.of.flight
- Broad.phase.of.flight

Inspect and identify potential cleaning tasks in each of the above columns. Execute those cleaning tasks. 

**Note**: You do not necessarily need to impute or drop NaNs here.

In [404]:
# Engine.Type

# Column to analyze
col_name = 'Engine.Type'

# Count occurrences of each unique value sorted by the value
value_counts = df[col_name].value_counts().sort_index()

# sort by counts in descending order
#value_counts = df[col_name].value_counts().sort_values(ascending=False)

# Print each value and its count
for value, count in value_counts.items():
    print(f"{value}: {count}")

Electric: 5
Geared Turbofan: 12
Reciprocating: 15015
Turbo Fan: 929
Turbo Jet: 133
Turbo Prop: 1251
Turbo Shaft: 11
UNK: 1
Unknown: 134


In [405]:
# Replace unknowns and nones with NaN
df['Engine.Type'] = df['Engine.Type'].replace(['UNK', 'Unknown'], np.nan)

# Check results
print(df[['Engine.Type']].head(10))


     Engine.Type
0      Turbo Fan
1      Turbo Fan
2  Reciprocating
3     Turbo Prop
4      Turbo Fan
5  Reciprocating
6     Turbo Prop
7      Turbo Fan
8  Reciprocating
9  Reciprocating


In [406]:
# Weather.Condition

# Column to analyze
col_name = 'Weather.Condition'

# Count occurrences of each unique value sorted by the value
value_counts = df[col_name].value_counts().sort_index()

# sort by counts in descending order
#value_counts = df[col_name].value_counts().sort_values(ascending=False)

# Print each value and its count
for value, count in value_counts.items():
    print(f"{value}: {count}")

IMC: 1069
UNK: 95
Unk: 215
VMC: 17089


In [407]:
# Replace unknowns with NaN
df['Weather.Condition'] = df['Weather.Condition'].str.upper().replace(['UNK'], np.nan)
                             
# Check results
print(df['Weather.Condition'].head(10))

0    VMC
1    VMC
2    IMC
3    VMC
4    VMC
5    VMC
6    VMC
7    VMC
8    VMC
9    VMC
Name: Weather.Condition, dtype: object


In [408]:
# Number.of.Engines

# Column to analyze
col_name = 'Number.of.Engines'

# Count occurrences of each unique value sorted by the value
value_counts = df[col_name].value_counts().sort_index()

# sort by counts in descending order
#value_counts = df[col_name].value_counts().sort_values(ascending=False)

# Print each value and its count
for value, count in value_counts.items():
    print(f"{value}: {count}")

0.0: 9
1.0: 15742
2.0: 3010
3.0: 40
4.0: 93
6.0: 1
8.0: 1


In [409]:
# Replace 0 with NaN
df['Number.of.Engines'] = df['Number.of.Engines'].replace([0], np.nan)
                             
# Check results
print(df['Number.of.Engines'].head(10))

0    3.0
1    4.0
2    1.0
3    2.0
4    3.0
5    2.0
6    2.0
7    3.0
8    1.0
9    1.0
Name: Number.of.Engines, dtype: float64


In [410]:
# Purpose.of.flight

# Column to analyze
col_name = 'Purpose.of.flight'

# Count occurrences of each unique value sorted by the value
value_counts = df[col_name].value_counts().sort_index()

# sort by counts in descending order
#value_counts = df[col_name].value_counts().sort_values(ascending=False)

# Print each value and its count
for value, count in value_counts.items():
    print(f"{value}: {count}")

ASHO: 5
Aerial Application: 935
Aerial Observation: 161
Air Drop: 3
Air Race show: 48
Air Race/show: 4
Banner Tow: 89
Business: 499
Executive/corporate: 103
External Load: 1
Ferry: 103
Firefighting: 17
Flight Test: 120
Glider Tow: 35
Instructional: 2753
Other Work Use: 155
PUBS: 3
Personal: 11714
Positioning: 350
Public Aircraft: 39
Public Aircraft - Federal: 52
Public Aircraft - Local: 12
Public Aircraft - State: 24
Skydiving: 164
Unknown: 378


In [411]:
# Replace Unknown with NaN, combine the "Air Race show" values
df['Purpose.of.flight'] = df['Purpose.of.flight'].replace(['Unknown'], np.nan).replace('Air Race/show', 'Air Race show')
                             
# Check results
print(df['Purpose.of.flight'].head(10))

0          NaN
1          NaN
2     Personal
3    Skydiving
4          NaN
5    Skydiving
6          NaN
7          NaN
8     Personal
9     Personal
Name: Purpose.of.flight, dtype: object


In [ ]:
# Broad.phase.of.flight

# Column to analyze
col_name = 'Broad.phase.of.flight'

# Count occurrences of each unique value sorted by the value
value_counts = df[col_name].value_counts().sort_index()

# sort by counts in descending order
#value_counts = df[col_name].value_counts().sort_values(ascending=False)

# Print each value and its count
for value, count in value_counts.items():
    print(f"{value}: {count}")

Approach: 234
Climb: 60
Cruise: 266
Descent: 73
Go-around: 89
Landing: 1255
Maneuvering: 169
Other: 3
Standing: 47
Takeoff: 509
Taxi: 111
Unknown: 12


In [413]:
# Replace Unknown with NaN
df['Broad.phase.of.flight'] = df['Broad.phase.of.flight'].replace(['Unknown'], np.nan).replace(['Unknown', 'Other'], np.nan)
                             
# Check results
print(df['Broad.phase.of.flight'].head(10))

0    Standing
1        Taxi
2      Cruise
3    Standing
4      Cruise
5         NaN
6       Climb
7        Taxi
8       Climb
9     Takeoff
Name: Broad.phase.of.flight, dtype: object


### Column Removal
- inspect the dataframe and drop any columns that have too many NaNs

In [418]:
# Calculate the total number of NaN values per field, and the percentage of NaN values per field

# Total number of records
total_records = len(df)

# Create dataframe with nan_stats including count and percent of NaNs
nan_stats = pd.DataFrame({
    'NaN_Count': df.isna().sum(),
    'non-Nan_Count': df.notna().sum(),
    'NaN_Percentage': (df.isna().sum() / total_records) * 100
})

# Sort by highest percentage NaNs
nan_stats = nan_stats.sort_values(by='NaN_Percentage', ascending=False)

print(nan_stats)

                        NaN_Count  non-Nan_Count  NaN_Percentage
Broad.phase.of.flight       18634           2813       86.883946
Engine.Type                  4091          17356       19.074929
Purpose.of.flight            4058          17389       18.921061
Weather.Condition            3289          18158       15.335478
Number.of.Engines            2560          18887       11.936401
Aircraft.damage              1324          20123        6.173358
Publication.Date              957          20490        4.462163
Injury.Severity               813          20634        3.790740
FAR.Description               480          20967        2.238075
Registration.Number           206          21241        0.960507
Location                        6          21441        0.027976
Country                         1          21446        0.004663
Total.Serious.Injuries          0          21447        0.000000
Total.Passengers                0          21447        0.000000
Total.Uninjured          

In [417]:
# # Dropping fields that have more than 20000 NaN values EXCEPT for those that were cleaned above and could be important to analysis
df = df.drop(columns=['Schedule', 'Air.carrier', 'Airport.Code', 'Airport.Name','Report.Status', 'Longitude','Latitude'])

# PRint columns
print(df.columns)

Index(['Event.Id', 'Investigation.Type', 'Accident.Number', 'Event.Date',
       'Location', 'Country', 'Injury.Severity', 'Aircraft.damage',
       'Aircraft.Category', 'Registration.Number', 'Make', 'Model',
       'Amateur.Built', 'Number.of.Engines', 'Engine.Type', 'FAR.Description',
       'Purpose.of.flight', 'Total.Fatal.Injuries', 'Total.Serious.Injuries',
       'Total.Minor.Injuries', 'Total.Uninjured', 'Weather.Condition',
       'Broad.phase.of.flight', 'Publication.Date', 'Total.Passengers',
       'Aircraft.destroyed', 'Make.Model'],
      dtype='object')


### Save DataFrame to csv
- its generally useful to save data to file/server after its in a sufficiently cleaned or intermediate state
- the data can then be loaded directly in another notebook for further analysis
- this helps keep your notebooks and workflow readable, clean and modularized

In [420]:
df.to_csv('/Users/mikeb/Documents/Flatiron/AD02/lab/dsc-course0-m8-lab/data/AviationData_Cleaned.csv', index=False)